Utilizando o Python para criar um script de carregamento para a "Linha de Produção"

In [1]:
import pandas as pd

Mostrando o conteudo do CSV (Apenas os dois primeiros)

In [2]:
df = pd.read_csv('dim_linhas.csv')
df.head(2)

,id_linha,nome_linha,turno,capacidade_hora
0,1,Linha A,Manhã,1200
1,2,Linha A,Tarde,1200


Visualizando os metadados do nosso arquivo, para checagem de tipo de dados etc

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_linha         5 non-null      int64 
 1   nome_linha       5 non-null      object
 2   turno            5 non-null      object
 3   capacidade_hora  5 non-null      int64 
dtypes: int64(2), object(2)
memory usage: 292.0+ bytes


Primeiro modo, inserção manual, apenas cria um script de inserção.

In [21]:
with open('inserts_linha_producao.sql', 'w', encoding='utf-8') as f:
    for _, row in df.iterrows():
        nome_linha = row['nome_linha'].replace("'", "''")
        turno = row['turno'].replace("'", "''")
        
        insert_sql = (
            "INSERT INTO linha_producao (id_linha, nome_linha, turno, capacidade_hora) "
            "VALUES ({}, '{}', '{}', {});\n".format(row['id_linha'], nome_linha, turno, row['capacidade_hora'])
        )
        f.write(insert_sql)

print("Arquivo 'insert_linha_producao.sql' gerado com sucesso!")


Arquivo 'insert_linha_producao.sql' gerado com sucesso!


Segundo modo, inserção direta no banco de dados, executa os comandos de insert diretamente.

In [5]:
# Primeiramente, vou utilizar uma biblioteca para conectar ao MySQL e inserir os dados do DataFrame.

%pip install mysql-connector-python
import mysql.connector

# Função para inserir dados no MySQL, definindo o nome da tabela como 'linha_producao' e a conexão com o banco de dados.
def insert_data_to_mysql(df, table_name='linha_producao'):
    connection_params = {
        'host': 'localhost',
        'user': 'root',
        'password': 'pw4dummy',
        'database': 'prod_nsk'
    }
    data_to_insert = []  # declarar fora do try para retornar depois

    try:
        # Conectar ao banco de dados MySQL
        conn = mysql.connector.connect(**connection_params)
        # criar um cursor para executar comandos SQL
        cursor = conn.cursor()

        # SQL para inserir dados na tabela
        # Usando f-string para formatar a consulta SQL com o nome da tabela
        sql = f"INSERT INTO {table_name} (id_linha, nome_linha, turno, capacidade_hora) VALUES (%s, %s, %s, %s)"

        # Iterar sobre o DataFrame e preparar os dados para inserção
        # Usando executemany para inserir múltiplas linhas de uma vez
        for _, row in df.iterrows():
            values = (
                int(row['id_linha']),
                row['nome_linha'],
                row['turno'],
                int(row['capacidade_hora'])
            )
            data_to_insert.append(values)

        cursor.executemany(sql, data_to_insert)
        conn.commit()

        #mensagem de sucesso
        print(f"{cursor.rowcount} registros inseridos com sucesso na tabela {table_name}.")
    
    except mysql.connector.Error as err:
        # mensagem de erro
        print(f"Erro MySQL: {err}")
    
    # fechar o cursor e a conexão
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
    
    return data_to_insert  # retornando aqui

# Executar e capturar os dados
data_to_insert = insert_data_to_mysql(df)



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   --------------- ------------------------ 6.6/16.4 MB 32.4 MB/s eta 0:00:01
   ------------------------------ --------- 12.3/16.4 MB 29.1 MB/s eta 0:00:01
   ---------------------------------------  16.3/16.4 MB 30.1 MB/s eta 0:00:01
   ---------------------------------------- 16.4/16.4 MB 23.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
5 registros inseridos com sucesso na tabela linha_producao.
